In [2]:
from fastai.vision import *

In [ ]:
def jac(input, target, eps = 1e-09):
    if torch.sum(target) == 0:
        return input.sum()*0.
    i_sig = torch.sigmoid(input)
    i_f = target.float()
    intersec = torch.sum(i_sig*i_f)
    union = torch.sum(i_sig+i_f-i_sig*i_f)
    return 1-(intersec/(union+eps))

def dic(input, target, eps = 1e-09):
    if torch.sum(target) == 0:
        return input.sum()*0.
    i_sig = torch.sigmoid(input)
    i_f = target.float()
    intersec = 2*torch.sum(i_sig*i_f)
    union = torch.sum(i_sig+i_f)
    return 1-((intersec+1)/(union+1))

In [12]:
def dic2(input, target):
    if torch.sum(target) == 0:
        return input.sum()*0.
    i_sig = torch.sigmoid(input)
    i_f = target.float()
    i_sig = i_sig.view(-1)
    i_f = i_f.view(-1)
    intersec = 2.*torch.sum(i_sig*i_f)
    union = i_sig.sum()+i_f.sum()
    return 1.-((intersec+1.)/(union+1.))

In [11]:
def weighted_dic(input, target):
    i_sig = torch.sigmoid(input)    
    i_f = target.float()
    w = torch.pow(i_f.sum(),2)+1e-09 # not sure about this
    intersec = 2*torch.sum(i_sig*i_f)/w
    union = torch.sum(i_sig+i_f)/w
    return 1.-((intersec+1.)/(union+1.))

In [9]:
def bcef(input, target):
    target = target.float()
    return torch.binary_cross_entropy_with_logits(input, target).mean()

In [8]:
class FocalLoss(nn.Module):
    def __init__(self, alpha = 0.25, gamma=2.):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, inputs, targets):
        loss = bcef(inputs, targets) # cross entropy
        loss = self.alpha * loss * (1 - torch.exp(-loss)) ** self.gamma # focal loss
        return loss

In [ ]:
def fl(input, target):
    loss = torch.binary_cross_entropy_with_logits(input, target.float())
    loss = loss * (1 - torch.exp(-loss))**.5
    return loss.mean()